In [41]:
!pip install pyarrow

In [31]:
import pandas as pd
df = pd.read_parquet("transformed_transactions.parquet")
df.head()

,id,cash_desk,total_amount,n_lines,payment_medium,has_feedback,feedback_low,feedback_middle,feedback_high,feedback_top,...,has_limited_time_offers,has_fruits_vegetables_pieces,has_beverages,has_long_shelf_life,has_missing,mean_time_between_scans,max_time_between_scans,time_to_first_scan,time_from_last_scan_to_end,days_since_sco_introduction
0,6f453598-6f80-4736-b808-c117204e9c71,2,75.99,9,CASH,False,False,False,False,False,...,False,True,True,False,False,9.086665,13.695168,9.933598,7.475138,479
1,25936480-7784-41d1-8054-11a075786bb0,2,70.71,8,CREDIT_CARD,False,False,False,False,False,...,False,True,True,False,False,7.049163,10.817932,11.008023,7.431704,479
2,e65dd2ec-b79c-4c4e-a6c3-cdc16ab230a0,2,57.84,4,CREDIT_CARD,False,False,False,False,False,...,False,True,False,False,False,3.558343,7.500713,4.877239,7.744029,479
3,f3977e82-c728-4ff3-b640-5cb985f9f1f3,2,7.94,2,CREDIT_CARD,False,False,False,False,False,...,False,False,False,True,False,3.646534,3.646534,1.499180,7.265990,479
4,5219ba05-0513-4cd3-a271-2f109ddccd1e,1,90.95,9,CASH,False,False,False,False,False,...,False,True,True,False,False,8.468498,13.634761,1.265051,11.473023,479


In [32]:
#abspeichern eines Ansichtsexemplars - liegt in meinem Ordner in Github
df.head(1000).to_csv("sample_transformed_transactions.csv", index=False)

In [33]:
# Hilfsfunktion zur Umlaut-Ersetzung
def replace_umlauts(text):
    if isinstance(text, str):
        return (text.replace("ä", "ae")
                    .replace("ö", "oe")
                    .replace("ü", "ue")
                    .replace("Ä", "Ae")
                    .replace("Ö", "Oe")
                    .replace("Ü", "Ue")
                    .replace("ß", "ss"))
    return text

# Ersetze Umlaute in location
df['location'] = df['location'].apply(replace_umlauts)

In [37]:
# Ersetze Monatszahlen durch Monatsnamen
month_map = {
    1: "January", 2: "February", 3: "March", 4: "April",
    5: "May", 6: "June", 7: "July", 8: "August",
    9: "September", 10: "October", 11: "November", 12: "December"
}
df['month'] = pd.to_numeric(df['month'], errors='coerce')
df['month'] = df['month'].map(month_map)

In [38]:
df['month']

0          May
1          May
2          May
3          May
4          May
          ... 
1481778    May
1481779    May
1481780    May
1481781    May
1481782    May
Name: month, Length: 1481783, dtype: object

In [39]:
# -------------------------------
# DAMAGE-Version
# -------------------------------
df_damage = df[df['damage'].notna()].drop(columns=['id'])
columns_with_damage_first = ['damage'] + [col for col in df_damage.columns if col not in ['damage', 'label']]
df_damage_first = df_damage[columns_with_damage_first]
df_damage_first.to_parquet("transformed_damage_first.parquet", index=False)

In [42]:

# -------------------------------
# LABEL-Version
# -------------------------------
df_label = df[df['label'].isin(['FRAUD', 'NORMAL'])].drop(columns=['id'])
columns_with_label_first = ['label'] + [col for col in df_label.columns if col not in ['damage', 'label']]
df_label_first = df_label[columns_with_label_first]
df_label_first.to_parquet("transformed_label_first.parquet", index=False)

In [44]:
display(df_damage_first.head(10))

,damage,cash_desk,total_amount,n_lines,payment_medium,has_feedback,feedback_low,feedback_middle,feedback_high,feedback_top,...,has_limited_time_offers,has_fruits_vegetables_pieces,has_beverages,has_long_shelf_life,has_missing,mean_time_between_scans,max_time_between_scans,time_to_first_scan,time_from_last_scan_to_end,days_since_sco_introduction
42,0.0,0,56.45,2,CREDIT_CARD,False,False,False,False,False,...,False,True,False,False,False,9.205984,9.205984,2.601115,6.628370,479
49,0.0,3,149.73,25,CREDIT_CARD,False,False,False,False,False,...,False,True,True,True,False,5.579174,13.259153,1.794106,10.571143,479
52,0.0,0,38.33,3,CREDIT_CARD,False,False,False,False,False,...,False,False,True,False,False,8.046765,14.779962,1.183578,4.660375,479
60,0.0,0,57.54,7,CREDIT_CARD,False,False,False,False,False,...,False,True,True,False,False,6.438000,10.540794,7.088195,8.478462,479
61,0.0,2,50.90,8,CREDIT_CARD,False,False,False,False,False,...,False,True,True,False,False,6.887883,12.634943,1.037482,9.131956,479
77,0.0,0,267.51,25,CREDIT_CARD,True,False,False,False,True,...,False,True,True,False,False,7.342497,16.359716,7.116740,17.909933,479
95,0.0,0,155.52,22,CREDIT_CARD,False,False,False,False,False,...,False,True,True,True,False,6.425951,14.351451,7.948612,12.095884,479
111,0.0,2,35.31,4,CREDIT_CARD,False,False,False,False,False,...,False,True,True,False,False,6.078169,10.406037,7.029285,14.999726,479
124,0.0,0,63.14,3,CREDIT_CARD,False,False,False,False,False,...,False,False,True,False,False,3.544712,5.002050,4.077807,10.656555,479
152,0.0,1,115.25,12,CREDIT_CARD,True,False,False,False,True,...,False,True,True,False,False,6.893364,15.111831,1.674608,16.217764,479


In [45]:
display(df_label_first.head(10))

,label,cash_desk,total_amount,n_lines,payment_medium,has_feedback,feedback_low,feedback_middle,feedback_high,feedback_top,...,has_limited_time_offers,has_fruits_vegetables_pieces,has_beverages,has_long_shelf_life,has_missing,mean_time_between_scans,max_time_between_scans,time_to_first_scan,time_from_last_scan_to_end,days_since_sco_introduction
42,NORMAL,0,56.45,2,CREDIT_CARD,False,False,False,False,False,...,False,True,False,False,False,9.205984,9.205984,2.601115,6.628370,479
49,NORMAL,3,149.73,25,CREDIT_CARD,False,False,False,False,False,...,False,True,True,True,False,5.579174,13.259153,1.794106,10.571143,479
52,NORMAL,0,38.33,3,CREDIT_CARD,False,False,False,False,False,...,False,False,True,False,False,8.046765,14.779962,1.183578,4.660375,479
60,NORMAL,0,57.54,7,CREDIT_CARD,False,False,False,False,False,...,False,True,True,False,False,6.438000,10.540794,7.088195,8.478462,479
61,NORMAL,2,50.90,8,CREDIT_CARD,False,False,False,False,False,...,False,True,True,False,False,6.887883,12.634943,1.037482,9.131956,479
77,NORMAL,0,267.51,25,CREDIT_CARD,True,False,False,False,True,...,False,True,True,False,False,7.342497,16.359716,7.116740,17.909933,479
95,NORMAL,0,155.52,22,CREDIT_CARD,False,False,False,False,False,...,False,True,True,True,False,6.425951,14.351451,7.948612,12.095884,479
111,NORMAL,2,35.31,4,CREDIT_CARD,False,False,False,False,False,...,False,True,True,False,False,6.078169,10.406037,7.029285,14.999726,479
124,NORMAL,0,63.14,3,CREDIT_CARD,False,False,False,False,False,...,False,False,True,False,False,3.544712,5.002050,4.077807,10.656555,479
152,NORMAL,1,115.25,12,CREDIT_CARD,True,False,False,False,True,...,False,True,True,False,False,6.893364,15.111831,1.674608,16.217764,479


In [49]:
#Erste 1000 Zeilen als Ansichtsexemplar in csv Format
df_label_first.head(1000).to_csv("sample_label_first.csv", index=False)
df_damage_first.head(1000).to_csv("sample_damage_first.csv", index=False)

In [51]:
df_label_first["label"].unique

<bound method Series.unique of 42         NORMAL
49         NORMAL
52         NORMAL
60         NORMAL
61         NORMAL
            ...  
1481748    NORMAL
1481753    NORMAL
1481755    NORMAL
1481761    NORMAL
1481771    NORMAL
Name: label, Length: 148025, dtype: category
Categories (3, object): ['FRAUD', 'UNKNOWN', 'NORMAL']>